# Explore dump with mysql connect

## Imports and paths

In [68]:
import os
from pathlib import Path
import pandas as pd
import psycopg2
import mysql.connector as mysql

import subprocess

In [69]:
home_path = str(Path.home())
os.chdir(home_path + '/git/predwikt/notebooks')

## Starting / Restarting MariaDB after it dies:

### Initialize in bash

#### Initialize mysql manually in bash

**bash**
```bash
sudo service mysqld stop

sudo service mysqld start
```
(I had to use mysqld not mysql: [link](https://stackoverflow.com/questions/11990708/error-cant-connect-to-local-mysql-server-through-socket-var-run-mysqld-mysq))

#### Check sql is running

**bash**
```bash
netstat -lnp | grep mysql
```

### Starting mysql.connect

#### Get passwords for mysql.connect

In [92]:
mysql_superuser = 'root'
# mysql_su_pass = input(f'Enter the MySQL password for user {mysql_superuser}: ')

In [66]:
pg_superuser = 'postgres'
# pg_su_pass = input(f'Enter the PostgreSQL password for user {pg_superuser}: ')

In [90]:
mysql_defaultuser = 'bhrdwj'
# mysql_def_pass = input(f'Enter the MySQL password for user {mysql_defaultuser}: ')
pg_defaultuser = 'bhrdwj'
# pg_def_pass = input(f'Enter the PostgreSQL password for user {pg_defaultuser}: ')

#### Connect mysql.connect to mysql

In [112]:
db_mysql = mysql.connect(
    host = "localhost",
    user = "root",
    passwd = mysql_su_pass,
    database='latest_page' # not the first time around
)

#### Initialize a cursor

In [113]:
cur_mysql = db_mysql.cursor()

#### Check current user

In [114]:
cur_mysql.execute('select user();')
cur_mysql.fetchall()

[('root@localhost',)]

In [115]:
cur_mysql.execute('select database();')
cur_mysql.fetchall()

[('latest_page',)]

In [120]:
cur_mysql.execute('show tables;')
cur_mysql.fetchall()

[('page',)]

In [119]:
cur_mysql.execute('DESCRIBE page;')
schema = pd.DataFrame(cur_mysql.fetchall(), columns=['Field','Type','Null','Key','Default','Extra'])
schema;

In [67]:
# schema.to_csv('../data/raw/schemas/schema_enwiki-latest-page.csv')

### Check if import finished

#### Look for first record of table

In [98]:
cur_mysql.execute("select * from page where page_title = 'AccessibleComputing';")
cur_mysql.fetchall()

[(10,
  0,
  bytearray(b'AccessibleComputing'),
  b'',
  1,
  0,
  0.33167112649574004,
  bytearray(b'20211031215759'),
  bytearray(b'20210715014751'),
  1002250816,
  111,
  bytearray(b'wikitext'),
  None)]

#### Look for one of the last records of table

In [88]:
cur_mysql.execute("select * from page where page_title = 'Traveler_Mijan';")
cur_mysql.fetchall()

[]

Import hadn't finished when it timed out during import.

## Not-repeated database setup

### Import dump into MySQL manually in bash

#### Created the database

In [36]:
# cur_mysql.execute("CREATE DATABASE latest_page CHARACTER SET utf8 COLLATE utf8_bin;")

#### Imported database from sql dump manually in bash

**bash**
```bash
mysql --user=bhrdwj --password=XXXXXXXX latest_page < enwiki-latest-page.sql > output.tab
```

### User privileges with mysql connect

#### Granted privileges to default user

In [103]:
# cur_mysql.execute(f"GRANT ALL PRIVILEGES ON latest_page.* TO 'bhrdwj'@'localhost' IDENTIFIED BY '{mysql_def_pass}';")

In [43]:
# cur_mysql.execute("FLUSH PRIVILEGES;")

In [40]:
cur_mysql.execute("SHOW GRANTS FOR 'bhrdwj'@'localhost';")
cur_mysql.fetchall()

[("GRANT USAGE ON *.* TO `bhrdwj`@`localhost` IDENTIFIED BY PASSWORD '*E5A65FB8301633C6247EC1EC461B78429C14BD14'",),
 ('GRANT ALL PRIVILEGES ON `latest_categ`.* TO `bhrdwj`@`localhost`',),
 ('GRANT ALL PRIVILEGES ON `latest_page`.* TO `bhrdwj`@`localhost`',)]

#### Switching user as necessary

In [155]:
# db_mysql.cmd_change_user(username=mysql_defaultuser, 
#                          password=mysql_def_pass,
#                          database='latest_page'
#                         )

## Whatever <mark>wouldn't work</mark> in mysql.connector yet

In [27]:
# the following isn't working :p

# subprocess.call(['sudo', 'service', '/usr/bin/mysql', 'start'], 
#                     shell=True, 
#                     stdin=subprocess.PIPE,
#                     stdout=subprocess.PIPE,
#                     stderr=subprocess.PIPE)

In [160]:
with open('../data/raw/enwiki-latest-page.sql', 'r') as f:
    for line in f:
        cur_mysql.execute(line, multi=True)
        db_mysql.commit()

In [153]:
with open('../data/raw/enwiki-latest-page.sql', 'r') as f:
    cur_mysql.execute(f.read(), multi=True)
    db_mysql.commit()

MemoryError: 

## <mark>Fix how mariadb keeps dying</mark>

https://askubuntu.com/questions/750604/why-does-mariadb-keep-dying-how-do-i-stop-it